In [ ]:
# Kill all processess on GPU
# !fuser -v /dev/nvidia* -k

# Libraries

In [11]:
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments, is_bf16_supported
import torch
from datetime import datetime
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Config

In [12]:
# Project configs
seed = 69
lang = 'id' # 'en' | 'id'
task = 'wikipedia' # 'wikipedia' | 'gsm8k'

# Data configs
train_size = 5000
test_size = 1000
max_seq_length = 1024
hf_data_id = 'wikimedia/wikipedia' # 'wikimedia/wikipedia' | 'openai/gsm8k'
hf_data_dir = '20231101.id' # 'wikipedia': '20231101.en' | '20231101.id' || 'gsm8k': 'main'
hf_data_split = f'train[:{(train_size+test_size)}]'

# Model configs
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# LoRA configs
lora_target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
lora_r = 8
lora_alpha = 16

resume_model_id = 'alxxtexxr/L3.1-8B-wikipedia-id-5K-LoRA-v20250628061607'
resume_from_checkpoint = bool(resume_model_id)
if resume_from_checkpoint:
    hub_model_id = resume_model_id
    project_name = hub_model_id.split('/')[-1]
    model_name = project_name

    from huggingface_hub import snapshot_download
    snapshot_download(repo_id=hub_model_id, local_dir=model_name)
else:
    model_name = 'unsloth/Meta-Llama-3.1-8B'
    project_name = f'L3.1-8B-{task}-{lang}-{train_size//1000}K-LoRA-v{datetime.now().strftime("%Y%m%d%H%M%S")}'
    hub_model_id = f'alxxtexxr/{project_name}'
print("Resume from checkpoint:", resume_from_checkpoint)
print("Project name:", project_name)
print("Hugging Face model ID:", hub_model_id)

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

adapter_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

adapter_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

...

trainer_state.json: 0.00B [00:00, ?B/s]

training_args.bin:   0%|          | 0.00/5.88k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Resume from checkpoint: True
Project name: L3.1-8B-wikipedia-id-5K-LoRA-v20250628061607
Hugging Face model ID: alxxtexxr/L3.1-8B-wikipedia-id-5K-LoRA-v20250628061607


# Model

In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.6.8: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.558 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    random_state=seed,
    target_modules=lora_target_modules,
    r=lora_r,
    lora_alpha=lora_alpha,   
    lora_dropout=0, # Supports any, but = 0 is optimized
    bias='none',    # Supports any, but = 'none' is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing=False, # True or 'unsloth' for very long context
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


# Data

In [15]:
dataset = load_dataset(hf_data_id, data_dir=hf_data_dir, split=hf_data_split)
eos_token = tokenizer.eos_token

def format_gsm8k_prompts(examples):
    gsm8k_prompt = """### Instruction:
Solve the following math problem step by step.

### Question: 
{question}

### Answer: 
{answer}""" + eos_token
    
    return {'text': [gsm8k_prompt.format(question=question, answer=answer) for question, answer in zip(examples['question'], examples['answer'])]}

def format_prompts(examples):
    return {'text': [example + eos_token for example in examples['text']]}

if task == 'gsm8k':
    dataset = dataset.map(format_gsm8k_prompts, batched=True)
else:
    dataset = dataset.map(format_prompts, batched=True)

In [16]:
dataset_split = dataset.train_test_split(train_size=train_size, test_size=test_size, seed=seed)
print(dataset_split)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1000
    })
})


In [17]:
# Sanity check
for row in dataset_split['train'][:3]["text"]:
    print("================================================================")
    print(row)

Kerajaan Sealand adalah sebuah bangsa mikro yang terletak di tengah-tengah Pantai Suffolk, Inggris. Sejak tahun 1967, wilayah ini ditempati oleh seorang mantan Mayor Inggris Paddy Roy Bates. Akan tetapi sejak 2012, kepemimpinan Sealand digantikan oleh anaknya yaitu Michael Bates. Keluarganya mengklaim tempat ini dan mengganti bendera sealand yang lama menjadi bendera sealand baru yang bergambar burung laut dan sebagai negara berdaulat yang independen.Kerajaan ini memiliki sumber daya alam yang melimpah sehingga banyak negara asing yang ingin menaklukkan negara tersebut sehingga negara ini dapat ditaklukkan oleh kerajaan Britania Raya karena sumber daya alam yang melimpah dan menjadi salah satu wilayah persemakmuran alam Britania raya == Legal status ==

In 1987, the UK extended its territorial waters from . Sealand is now in British territorial waters. In the opinion of law academic John Gibson, there is little to no chance that Sealand would be recognised as a nation due to it being a

# Training

In [18]:
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_split['train'],
    # eval_dataset=dataset_split['test'],
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    dataset_num_proc=8,

    args=TrainingArguments(
        seed=seed,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        # max_steps=3, # For debugging
        warmup_ratio=0.05,
        learning_rate=2e-4,
        lr_scheduler_type='cosine',
        optim='paged_adamw_8bit', # 'paged_adamw_8bit' | 'adamw_8bit'
        weight_decay=0.00,
        max_grad_norm=0.3,
        fp16=(not is_bf16_supported()),
        bf16=is_bf16_supported(),

        # Eval arguments
        # eval_strategy='steps',
        # eval_steps=10,
        
        # Logging arguments
        logging_strategy='steps',
        logging_steps=1,
        # logging_first_step=True,
        report_to=['tensorboard', 'wandb'],

        # Saving arguments
        save_strategy='steps',
        save_steps=50,
        # save_steps=1, # For debugging
        save_total_limit=5, # 1 best + 4 recent checkpoints. Warning: It doesn't work
        
        # With load_best_model_at_end=True, your save_strategy will be ignored and default to eval_strategy.
        # So you will find one checkpoint at the end of each epoch.
        # https://discuss.huggingface.co/t/trainer-not-saving-after-save-steps/5464
        # load_best_model_at_end=True, 

        output_dir=project_name,
        hub_model_id=hub_model_id,
        push_to_hub=True,

        hub_strategy='all_checkpoints',
        hub_always_push=True,
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [19]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A5000. Max memory = 23.558 GB.
13.309 GB of memory reserved.


In [20]:
# Start training
trainer_stats = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 3 | Total steps = 1,875
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 20,971,520/8,000,000,000 (0.26% trained)


Step,Training Loss
